In [118]:
import sys
sys.path.append("../../helios-server/")
import helios.models as models
from helios_auth import models as auth_model
import helios.views as views
from helios.models import CastVote, Voter, Trustee
from helios.workflows.homomorphic import EncryptedVote, Tally
from election_utils import *
from tqdm import tqdm, trange
# import click
import time
import os
import matplotlib.pyplot as plt
import numpy as np

In [127]:
num_questions = 4
num_choices = 4
num_trustees = 1
num_voters = 1

In [128]:
election = models.Election.get_by_short_name(f'election_{num_questions}_questions_{num_choices}_choices_{num_trustees}_trustees_{num_voters}_voters')

In [129]:
if election is None:
    os.system(f"python prep_election.py -q {num_questions} -c {num_choices} -v {num_voters}")
election = models.Election.get_by_short_name(f'election_{num_questions}_questions_{num_choices}_choices_{num_trustees}_trustees_{num_voters}_voters')

In [130]:
election

<Election: Election object (1)>

In [131]:
answers = prepare_answers(num_questions, num_choices)

In [132]:
answers

[[1, 3, 0], [1, 2], [1, 0, 2], [0]]

In [133]:
tally = Tally(election=election)
vote = EncryptedVote.fromElectionAndAnswers(election, answers)

stops = [10**i for i in range(1, 7)]
votes = []
times = []

for i in trange(1, stops[-1]+1):
    tally.add_vote(vote, verify_p=False)
    if i in stops:
        election.encrypted_tally = tally
        election.helios_trustee_decrypt()
        trustees = Trustee.get_by_election(election)
        decryption_factors = [t.decryption_factors for t in trustees]

        tic = time.perf_counter()
        result = tally.decrypt_from_factors(decryption_factors, election.public_key)

        toc = time.perf_counter()
        decrypt_time = toc - tic
        print(f"#Votes: {i}, Tally-Time: {decrypt_time}")
        votes.append(i)
        times.append(decrypt_time)
        # print(result)
        
np.save(f"timing_results/decryption_times_{votes[-1]}V_{num_questions}Q_{num_choices}C.npy", [votes, times])

  0%|          | 10/1000000 [00:03<110:19:35,  2.52it/s]

#Votes: 10, Tally-Time: 0.02122335508465767


  0%|          | 385/1000000 [00:05<1:54:45, 145.18it/s]

#Votes: 100, Tally-Time: 0.022913534194231033


  0%|          | 1216/1000000 [00:06<29:18, 568.12it/s] 

#Votes: 1000, Tally-Time: 0.04175592586398125


  1%|          | 10210/1000000 [00:12<25:09, 655.91it/s]

#Votes: 10000, Tally-Time: 0.233208529651165


  7%|▋         | 65224/1000000 [00:51<10:58, 1419.28it/s]

In [89]:
tally.add_vote(vote, verify_p=False)

In [90]:
tally.num_tallied

1

In [94]:
%%time
# gather the decryption factors
election.encrypted_tally = tally
election.helios_trustee_decrypt()
trustees = Trustee.get_by_election(election)
decryption_factors = [t.decryption_factors for t in trustees]
result = tally.decrypt_from_factors(decryption_factors, election.public_key)

CPU times: user 109 ms, sys: 51 µs, total: 109 ms
Wall time: 110 ms


In [92]:
tally

In [95]:
result

[[0, 1, 1, 1]]

In [40]:
vote.encrypted_answers[0].choices[0].__dict__

{'pk': <helios.crypto.elgamal.PublicKey at 0x7f1ac6bcb8b0>,
 'alpha': 280004405091145374651889405264302285081934236931410147033432346617204077776908073635919961513827794442541035801908763272058776562934892963437873812429502053859626295589900565685583259446644991387364649447783407615899897564915153904348140675302615674888510475137552744421674111560058890849955630814445082131728562955102586914130200434580800588737935924566128527999901433416309331993541718937139699604183490311212019870721296806845435742431841948282216116650061445389511167564400321825347638299834236376024385640139343432660573419068174082091650812749276739730266434346412673161994495142154981283179281629543247206889,
 'beta': 1581584265949190596665586894062098463938650995756197918743450684179888298441143487534015529199460884691922231381028485005346963959848992887076683307311021485311082012113747673945436567436217678216175500410966021441146393366174277244639264311308616393061268710796403449528475432395221401155890691046357987

In [ ]:
# election.compute_tally(verify_p=True)

In [30]:
vote

In [41]:
castvote = CastVote(vote=vote, vote_hash=vote.hash, voter=None)

In [50]:
castvote.vote.encrypted_answers[0].choices[0]